In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

import os
import sys

import tensorflow as tf 

import yfinance as yf 

#    Fetching the stock data

Price and volume

In [2]:
data = yf.Ticker("AAPL")

ohlc = data.history(period="max", interval="1d")[['Open', 'High', 'Low', 'Close', 'Volume']]
ohlc.index = ohlc.index.tz_localize(None)  # Make index timezone-naive
volume = ohlc['Volume']

Latest quaterly financials 

In [3]:
dividends = data.dividends
splits = data.splits
actions = data.actions
capital_gains = data.capital_gains

income_stmt = data.income_stmt
balance_sheet = data.balance_sheet
cashflow = data.cash_flow
tmm_cashflow = data.ttm_cash_flow
earnings = data.earnings

c:\Users\dawid\Desktop\BSEPrediction\.venv\Lib\site-packages\yfinance\scrapers\fundamentals.py:36: DeprecationWarning: 'Ticker.earnings' is deprecated as not available via API. Look for "Net Income" in Ticker.income_stmt.
  warnings.warn("'Ticker.earnings' is deprecated as not available via API. Look for \"Net Income\" in Ticker.income_stmt.", DeprecationWarning)


Additional data

In [4]:
info = data.info
fast_info = data.fast_info
calendar = data.calendar
earnings_dates = data.earnings_dates
# sec_fillings = data.sec_fillings

Analysis 

In [5]:
recommendations = data.recommendations
recommendations_summary = data.recommendations_summary

upgrades_downgrades = data.upgrades_downgrades
sustainability = data.sustainability

analyst_price_targets = data.analyst_price_targets
earnings_estimate = data.earnings_estimate
revenue_estimate = data.revenue_estimate

earnings_history = data.earnings_history
eps_trend = data.eps_trend
# eps_revision = data.eps_revision

Holdings

In [6]:
funds_data = data.funds_data
insider_purchases = data.insider_purchases
insider_transactions = data.insider_transactions
insider_roster_holders = data.insider_roster_holders
major_holders = data.major_holders
# institutional_holder = data.institutional_holder
mutualfund_holders = data.mutualfund_holders

# Fetching the market data

In [7]:
us_market = yf.Market('US')
gb_market = yf.Market('GB')

asian_market = yf.Market('ASIA')
european_market = yf.Market('EUROPE')

rates = yf.Market('RATES')
commodities = yf.Market('COMMODITIES')
currencies = yf.Market('CURRENCIES')
crypto = yf.Market('CRYPTOCURRENCIES')

# Sector and industry

In [8]:
print(yf.Sector.industries) # to be continued

# Data cleaning and preprocessing

In [9]:
total_dividend_through_years = dividends.sum()
total_number_of_stock_splits = len(splits)
# Process capital gains 

Financial statement transformation

In [10]:
final_df = pd.DataFrame(index=ohlc.index)

# Income Statement
income_stmt = data.financials.T  # Transpose so each row is a report
income_stmt.index = pd.to_datetime(income_stmt.index)  # Ensure datetime index

income_stmt_sorted = income_stmt.sort_index()
aligned_income = pd.DataFrame(index=ohlc.index, columns=income_stmt.columns)

for report_date in income_stmt_sorted.index:
    mask = (aligned_income.index >= report_date)
    aligned_income.loc[mask] = aligned_income.loc[mask].fillna(method='ffill')
    aligned_income.loc[aligned_income.index >= report_date] = \
    pd.DataFrame([income_stmt.loc[report_date].values] * len(aligned_income.loc[aligned_income.index >= report_date]),
                 index=aligned_income.loc[aligned_income.index >= report_date].index,
                 columns=aligned_income.columns)
    
aligned_income.fillna(method='ffill', inplace=True)

# Balance Sheet
balance_sheet = data.balance_sheet.T
balance_sheet.index = pd.to_datetime(balance_sheet.index)
balance_sheet_sorted = balance_sheet.sort_index()
aligned_balance = pd.DataFrame(index=ohlc.index, columns=balance_sheet.columns)
for report_date in balance_sheet_sorted.index:
    mask = (aligned_balance.index >= report_date)
    aligned_balance.loc[mask] = aligned_balance.loc[mask].fillna(method='ffill')
    aligned_balance.loc[aligned_balance.index >= report_date] = \
        pd.DataFrame([balance_sheet.loc[report_date].values] * len(aligned_balance.loc[aligned_balance.index >= report_date]),
                     index=aligned_balance.loc[aligned_balance.index >= report_date].index,
                     columns=aligned_balance.columns)
aligned_balance.fillna(method='ffill', inplace=True)

# Cash Flow
cashflow = data.cashflow.T
cashflow.index = pd.to_datetime(cashflow.index)
cashflow_sorted = cashflow.sort_index()
aligned_cashflow = pd.DataFrame(index=ohlc.index, columns=cashflow.columns)
for report_date in cashflow_sorted.index:
    mask = (aligned_cashflow.index >= report_date)
    aligned_cashflow.loc[mask] = aligned_cashflow.loc[mask].fillna(method='ffill')
    aligned_cashflow.loc[aligned_cashflow.index >= report_date] = \
        pd.DataFrame([cashflow.loc[report_date].values] * len(aligned_cashflow.loc[aligned_cashflow.index >= report_date]),
                     index=aligned_cashflow.loc[aligned_cashflow.index >= report_date].index,
                     columns=aligned_cashflow.columns)
aligned_cashflow.fillna(method='ffill', inplace=True)

C:\Users\dawid\AppData\Local\Temp\ipykernel_19844\2446286178.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  aligned_income.loc[mask] = aligned_income.loc[mask].fillna(method='ffill')
C:\Users\dawid\AppData\Local\Temp\ipykernel_19844\2446286178.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  aligned_income.loc[mask] = aligned_income.loc[mask].fillna(method='ffill')
C:\Users\dawid\AppData\Local\Temp\ipykernel_19844\2446286178.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  aligned_income.loc[mask] = aligned_income.loc[mask].fillna(method='ffill')
C:\Users\dawid\AppData\Local\

In [11]:
# Dividends
# Align dividends to ohlc index (ensure both indexes are timezone-naive)
dividends = data.dividends.copy()
dividends.index = dividends.index.tz_localize(None)
aligned_dividends = pd.DataFrame(index=ohlc.index)
aligned_dividends['Dividends'] = dividends.reindex(ohlc.index, method='ffill').fillna(0)

# Splits
# Align splits to ohlc index (ensure both indexes are timezone-naive)
splits = data.splits.copy()
splits.index = splits.index.tz_localize(None)
aligned_splits = pd.DataFrame(index=ohlc.index)
aligned_splits['Splits'] = splits.reindex(ohlc.index, method='ffill').fillna(0)

# Combine all dataframes
combined_df = pd.concat([ohlc, aligned_income, aligned_balance, aligned_cashflow, aligned_dividends, aligned_splits], axis=1)

In [12]:
combined_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Tax Effect Of Unusual Items',
       'Tax Rate For Calcs', 'Normalized EBITDA',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation',
       ...
       'Changes In Account Receivables', 'Other Non Cash Items',
       'Stock Based Compensation', 'Deferred Tax', 'Deferred Income Tax',
       'Depreciation Amortization Depletion', 'Depreciation And Amortization',
       'Net Income From Continuing Operations', 'Dividends', 'Splits'],
      dtype='object', length=167)

Add features like technical indicators

# Model creation

Linear regression

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

model_df = combined_df.dropna()

X = model_df.drop(columns=["Close"])
y = model_df["Close"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr_regressor = LinearRegression()
lr_regressor.fit(X_train, y_train)

y_pred = lr_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Linear Regression Mean Squared Error: {mse:.2f}")
print(f"Linear Regression R^2 Score: {r2:.2f}")

Linear Regression Mean Squared Error: 1.44
Linear Regression R^2 Score: 1.00


Decision tree

In [20]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

model_df = combined_df.dropna()

X = model_df.drop(columns=["Close"])  
y = model_df["Close"] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

regressor = DecisionTreeRegressor(random_state=42)
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")

Mean Squared Error: 3.76
R^2 Score: 0.99


Random forest

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

model_df = combined_df.dropna()

X = model_df.drop(columns=["Close"])
y = model_df["Close"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

y_pred = rf_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Random Forest Mean Squared Error: {mse:.2f}")
print(f"Random Forest R^2 Score: {r2:.2f}")

Random Forest Mean Squared Error: 2.89
Random Forest R^2 Score: 0.99


XGBoost

In [16]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

model_df = combined_df.dropna()

X = model_df.drop(columns=["Close"])
y = model_df["Close"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_regressor = XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
xgb_regressor.fit(X_train, y_train)

y_pred = xgb_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Mean Squared Error: {mse:.2f}")
print(f"XGBoost R^2 Score: {r2:.2f}")

XGBoost Mean Squared Error: 4.15
XGBoost R^2 Score: 0.99


Support Vector Regression (SVR)

In [17]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

model_df = combined_df.dropna()
X = model_df.drop(columns=["Close"])
y = model_df["Close"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svr_regressor = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_regressor.fit(X_train, y_train)

y_pred = svr_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"SVR Mean Squared Error: {mse:.2f}")
print(f"SVR R^2 Score: {r2:.2f}")

SVR Mean Squared Error: 592.69
SVR R^2 Score: -0.04


K-Nearest Neighbors Regression (KNN)

In [18]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

model_df = combined_df.dropna()
X = model_df.drop(columns=["Close"])
y = model_df["Close"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn_regressor = KNeighborsRegressor(n_neighbors=5)
knn_regressor.fit(X_train, y_train)

y_pred = knn_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"KNN Mean Squared Error: {mse:.2f}")
print(f"KNN R^2 Score: {r2:.2f}")

KNN Mean Squared Error: 306.47
KNN R^2 Score: 0.46


LightGBM Regression

In [21]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

model_df = combined_df.dropna()
X = model_df.drop(columns=["Close"])
y = model_df["Close"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lgbm_regressor = LGBMRegressor(n_estimators=100, random_state=42)
lgbm_regressor.fit(X_train, y_train)

y_pred = lgbm_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"LightGBM Mean Squared Error: {mse:.2f}")
print(f"LightGBM R^2 Score: {r2:.2f}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000295 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 852
[LightGBM] [Info] Number of data points in the train set: 326, number of used features: 143
[LightGBM] [Info] Start training from score 206.940897
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

Neural Network Regression (MLP)

In [24]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

model_df = combined_df.dropna()
X = model_df.drop(columns=["Close"])
y = model_df["Close"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

mlp_regressor = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
mlp_regressor.fit(X_train, y_train)

y_pred = mlp_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MLP Mean Squared Error: {mse:.2f}")
print(f"MLP R^2 Score: {r2:.2f}")

MLP Mean Squared Error: 3095788080995789.00
MLP R^2 Score: -5410105662364.31


LSTM Neural Network Regression (Keras)

In [25]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

model_df = combined_df.dropna()
X = model_df.drop(columns=["Close"])
y = model_df["Close"]

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# Reshape for LSTM: (samples, timesteps, features)
# We'll use a window of 1 (no lookback)
X_lstm = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_scaled, test_size=0.2, random_state=42)

model = Sequential([
    LSTM(50, input_shape=(X_lstm.shape[1], X_lstm.shape[2])),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_inv = scaler_y.inverse_transform(y_test)

mse = mean_squared_error(y_test_inv, y_pred)
r2 = r2_score(y_test_inv, y_pred)

print(f"LSTM Mean Squared Error: {mse:.2f}")
print(f"LSTM R^2 Score: {r2:.2f}")

c:\Users\dawid\Desktop\BSEPrediction\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
LSTM Mean Squared Error: 29.34
LSTM R^2 Score: 0.95
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
LSTM Mean Squared Error: 29.34
LSTM R^2 Score: 0.95


ARIMA/SARIMA Time Series Forecasting

In [26]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score

# Use only the 'Close' price for time series forecasting
model_df = combined_df.dropna()
close_series = model_df['Close']

# Split into train and test sets (time series split, no shuffling)
split_idx = int(len(close_series) * 0.8)
train, test = close_series.iloc[:split_idx], close_series.iloc[split_idx:]

# Fit SARIMA model (simple order, can be tuned)
model = SARIMAX(train, order=(1,1,1), seasonal_order=(1,1,1,12))
model_fit = model.fit(disp=False)

# Forecast
forecast = model_fit.predict(start=test.index[0], end=test.index[-1], dynamic=False)

mse = mean_squared_error(test, forecast)
r2 = r2_score(test, forecast)

print(f"SARIMA Mean Squared Error: {mse:.2f}")
print(f"SARIMA R^2 Score: {r2:.2f}")

c:\Users\dawid\Desktop\BSEPrediction\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\dawid\Desktop\BSEPrediction\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


KeyError: 'The `start` argument could not be matched to a location related to the index of the data.'

Prophet Time Series Forecasting

In [ ]:
# If you don't have prophet installed, run: !pip install prophet
from prophet import Prophet
from sklearn.metrics import mean_squared_error, r2_score

model_df = combined_df.dropna()
close_series = model_df['Close']

# Prophet requires a DataFrame with columns 'ds' and 'y'
df_prophet = close_series.reset_index()
df_prophet.columns = ['ds', 'y']

# Ensure 'ds' is datetime and daily frequency
if not np.issubdtype(df_prophet['ds'].dtype, np.datetime64):
    df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])
df_prophet = df_prophet.sort_values('ds')
df_prophet = df_prophet.set_index('ds').asfreq('D').reset_index()

# Fill missing values (Prophet cannot handle NaN)
df_prophet['y'] = df_prophet['y'].interpolate(method='linear').fillna(method='bfill').fillna(method='ffill')

# Split into train and test sets
split_idx = int(len(df_prophet) * 0.8)
train, test = df_prophet.iloc[:split_idx], df_prophet.iloc[split_idx:]

model = Prophet()
model.fit(train)

future = model.make_future_dataframe(periods=len(test), freq='D')
forecast = model.predict(future)

# Align forecast with test set
y_pred = forecast['yhat'].iloc[-len(test):].values
mse = mean_squared_error(test['y'], y_pred)
r2 = r2_score(test['y'], y_pred)

print(f"Prophet Mean Squared Error: {mse:.2f}")
print(f"Prophet R^2 Score: {r2:.2f}")

Ensemble Method: Averaging Predictions from Multiple Models

In [ ]:
# Example: Average predictions from Linear Regression, Random Forest, and XGBoost
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

model_df = combined_df.dropna()
X = model_df.drop(columns=["Close"])
y = model_df["Close"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train models
lr = LinearRegression().fit(X_train, y_train)
rf = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)
xgb = XGBRegressor(n_estimators=100, random_state=42, verbosity=0).fit(X_train, y_train)

# Predict
y_pred_lr = lr.predict(X_test)
y_pred_rf = rf.predict(X_test)
y_pred_xgb = xgb.predict(X_test)

# Ensemble: simple average
y_pred_ensemble = (y_pred_lr + y_pred_rf + y_pred_xgb) / 3

mse = mean_squared_error(y_test, y_pred_ensemble)
r2 = r2_score(y_test, y_pred_ensemble)

print(f"Ensemble Mean Squared Error: {mse:.2f}")
print(f"Ensemble R^2 Score: {r2:.2f}")